In [ ]:
import pandas as pd
from scipy.stats import f_oneway, chi2_contingency
import pingouin as pg
import numpy as np
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.rcParams['font.family'] = 'Malgun Gothic'

In [4]:
YEAR = 2023
TARGET = 'customer'

In [5]:
PATH = f'../dataset/imbank_{TARGET.lower()}_data/iMBANK_{TARGET.upper()}_DATA_{YEAR}(K-DigitalTraining).csv'
df = pd.read_csv(PATH, encoding='euc-kr')

In [ ]:
# df1=df[['고객ID', '자택_시도', '연령대']]
# samp = df1.groupby('고객ID')['자택_시도'].unique().reset_index()
# # lambda x: True if len(x['자택_시도'])>1 else False
# # 자택 바뀐사람은 없음
# samp[samp['자택_시도'].apply(lambda x: True if len(x)>1 else False)]
# # df1.info # 총 200만명
# df1['자택_시도'].value_counts()
# df1=df1.drop_duplicates('고객ID')

In [ ]:
customer = df.groupby('고객ID')['기준년월'].nunique()

# 12개 이상의 기준년월을 가진 고객ID 필터링
customer = customer[customer == 12].index

# 필터링된 고객ID에 해당하는 데이터 추출
df_12 = df[df['고객ID'].isin(customer)]

df=df_12
del df_12

In [6]:
df['총자산'] = df[['수신_요구불예금', '수신_거치식예금', '수신_적립식예금', '수신_펀드', '수신_외화예금', '대출금액']].sum(axis=1)

In [7]:
# 분기 계산 (연도와 분기 번호 생성) 
df['분기'] = (df['기준년월'] % 100 - 1) // 3 + 1

In [8]:
df_age_q = df.groupby(['연령대', '분기'])['총자산'].mean().reset_index()

In [9]:
df= pd.merge(df, df_age_q, on=['연령대', '분기'], how='left')

In [10]:
df.columns = ['기준년월', '고객ID', '연령대', '성별', '고객등급', '자택_시도', '자택_시군구', '수신_요구불예금',
       '수신_거치식예금', '수신_적립식예금', '수신_펀드', '수신_외화예금', '대출금액', '총자산', '분기',
       '연령_분기별_자산평균']

# 분기 초/말의 총자산을 계산
df['분기_말_총자산'] = df.groupby(['고객ID', '분기'])['총자산'].transform('first')
df['분기_초_총자산'] = df.groupby(['고객ID', '분기'])['총자산'].transform('last')

# 분기 내 자산 변화량 계산
df['자산변화량'] = (df['분기_말_총자산'] - df['분기_초_총자산'])
df['자산변화율'] =  df['자산변화량'] / df['연령_분기별_자산평균']

In [11]:
df['연령_분기별_자산변화율_평균'] = df.groupby(['연령대', '분기'])['자산변화율'].transform('mean')

In [12]:
df=df[['기준년월', '고객ID', '연령대', '자택_시도', '분기', '자산변화량', '자산변화율', '총자산',
       '연령_분기별_자산변화율_평균']]

In [14]:
# 변화율에 따라 고객 분류
def classify_change_rate(change_rate):
    if change_rate > 10:
        return '초안전'
    elif change_rate > 0:
        return '안전'
    elif change_rate == 0:
        return '무관심'
    elif change_rate >= -10:
        return '위험'
    else:
        return '고위험'

In [15]:
df['연령_분기_자산변화량평균'] = df.groupby(['연령대', '분기'])['자산변화량'].transform('mean')

In [16]:
df['이탈여부'] = df['자산변화율'].apply(classify_change_rate)

In [17]:
november_data = df[df['기준년월'] % 100 == 11]
december_data = df[df['기준년월'] % 100 == 12]
df.loc[
    (df['고객ID'].isin(november_data[november_data['총자산'] == 0]['고객ID'])) &
    (df['고객ID'].isin(december_data[december_data['총자산'] == 0]['고객ID'])),
    '이탈여부'
] = '이탈'

In [16]:
df['이탈여부'].value_counts() 

이탈여부
안전     7156307
위험     5825377
무관심    4457916
이탈     4128979
초안전       5606
고위험       4501
Name: count, dtype: int64

In [18]:
df.to_csv(f'../dataset/iMB_{TARGET}_data_{YEAR}.csv', encoding='euc-kr', index=False)

In [19]:
df.columns

Index(['기준년월', '고객ID', '연령대', '자택_시도', '분기', '자산변화량', '자산변화율', '총자산',
       '연령_분기별_자산변화율_평균', '연령_분기_자산변화량평균', '이탈여부'],
      dtype='object')

In [ ]:
df = df[df['연령대'].isin(['20대', '30대'])]

In [ ]:
df['자택_시도'].unique()

array(['경상북도', '대구광역시', '충청북도', '부산광역시', '경기도', '서울특별시', '인천광역시', '경상남도',
       '울산광역시', '세종특별자치시', '제주특별자치도', '대전광역시', '강원특별자치도', '충청남도', '전라남도',
       '광주광역시', '전북특별자치도'], dtype=object)

In [ ]:
# 1. '지역구분' 컬럼을 0으로 초기화
df['지역구분'] = 0

# 2. '자택_시도'가 '대구광역시' 또는 '경상북도'인 경우 '지역구분'을 1로 설정
df.loc[df['자택_시도'].isin(['대구광역시', '경상북도']), '지역구분'] = 1

In [ ]:
df['성별'].value_counts()

성별
남    3285896
여    3066339
Name: count, dtype: int64

In [ ]:
df['지역구분'].value_counts()

지역구분
1    4716119
0    1636116
Name: count, dtype: int64

In [ ]:
df['연령대'].value_counts()

연령대
20대    3199166
30대    3153069
Name: count, dtype: int64

In [ ]:
df.drop(['자택_시도', '자택_시군구', '연령_분기별_자산평균', '분기_말_총자산', '분기_초_총자산', '자산변화량','연령_분기별_자산변화율_평균', '연령_분기_자산변화량평균'], axis = 1, inplace= True)

In [ ]:
df['대출여부'] = 0

# 2. '자택_시도'가 '대구광역시' 또는 '경상북도'인 경우 '지역구분'을 1로 설정
df.loc[df['대출금액']> 0 , '대출여부'] = 1

In [ ]:
df.to_csv(f'../dataset/imbank_{TARGET.lower()}_data_{YEAR}.csv', index=False)

In [ ]:
df['이탈여부'].unique()

array(['이탈', '위험', '무관심', '안전', '초안전', '고위험'], dtype=object)

In [ ]:
# ANOVA 실행
f_value, p_value = f_oneway(
    df[df['이탈여부'] == '이탈']['수신_요구불예금'],
    df[df['이탈여부'] == '위험']['수신_요구불예금'],
    df[df['이탈여부'] == '무관심']['수신_요구불예금'],
    df[df['이탈여부'] == '안전']['수신_요구불예금'],
    df[df['이탈여부'] == '초안전']['수신_요구불예금'],
    df[df['이탈여부'] == '고위험']['수신_요구불예금'],
)

# 결과 출력
print(f"F-value: {f_value}, P-value: {p_value}")

# p-value에 따라 결론 내기
if p_value < 0.05:
    print("그룹 간 평균에 유의미한 차이가 있습니다.")
else:
    print("그룹 간 평균에 유의미한 차이가 없습니다.")

F-value: 82659.78055242168, P-value: 0.0
그룹 간 평균에 유의미한 차이가 있습니다.


In [267]:
# # Tukey HSD 사후 검정
# tukey = pairwise_tukeyhsd(df_12['수신_요구불예금'], df_12['이탈여부'], alpha=0.05)

# # 결과 출력
# print(tukey.summary())

# # 결과를 시각적으로 확인 (중요)
# tukey.plot_simultaneous()
# plt.show()


In [268]:
# Welch's ANOVA
# Games-Howell 사후 검정
posthoc = pg.pairwise_gameshowell(dv='수신_요구불예금', between='이탈여부', data=df_12)
print(posthoc)

      A    B       mean(A)       mean(B)          diff             se  \
0   고위험  무관심  1.833165e+07  8.158599e+05  1.751579e+07  464812.674359   
1   고위험   안전  1.833165e+07  2.613907e+06  1.571774e+07  464829.711663   
2   고위험   위험  1.833165e+07  2.383516e+06  1.594813e+07  464829.652513   
3   고위험   이탈  1.833165e+07  7.565395e+04  1.825600e+07  464800.880691   
4   고위험  초안전  1.833165e+07  1.278193e+07  5.549716e+06  559660.409096   
5   무관심   안전  8.158599e+05  2.613907e+06 -1.798047e+06    6206.675070   
6   무관심   위험  8.158599e+05  2.383516e+06 -1.567656e+06    6202.243658   
7   무관심   이탈  8.158599e+05  7.565395e+04  7.402059e+05    3423.545259   
8   무관심  초안전  8.158599e+05  1.278193e+07 -1.196607e+07  311755.730469   
9    안전   위험  2.613907e+06  2.383516e+06  2.303906e+05    7369.289383   
10   안전   이탈  2.613907e+06  7.565395e+04  2.538253e+06    5249.691587   
11   안전  초안전  2.613907e+06  1.278193e+07 -1.016803e+07  311781.131694   
12   위험   이탈  2.383516e+06  7.565395e+04  2.307862e

In [ ]:
# ANOVA 실행
f_value, p_value = f_oneway(
    df[df['이탈여부'] == '이탈']['수신_거치식예금'],
    df[df['이탈여부'] == '위험']['수신_거치식예금'],
    df[df['이탈여부'] == '무관심']['수신_거치식예금'],
    df[df['이탈여부'] == '안전']['수신_거치식예금'],
    df[df['이탈여부'] == '초안전']['수신_거치식예금'],
    df[df['이탈여부'] == '고위험']['수신_거치식예금'],
)

# 결과 출력
print(f"F-value: {f_value}, P-value: {p_value}")

# p-value에 따라 결론 내기
if p_value < 0.05:
    print("그룹 간 평균에 유의미한 차이가 있습니다.")
else:
    print("그룹 간 평균에 유의미한 차이가 없습니다.")

F-value: 27276.62665618941, P-value: 0.0
그룹 간 평균에 유의미한 차이가 있습니다.


In [270]:
posthoc = pg.pairwise_gameshowell(dv='수신_거치식예금', between='이탈여부', data=df_12)
print(posthoc)


      A    B       mean(A)       mean(B)          diff             se  \
0   고위험  무관심  8.526810e+06  7.615902e+05  7.765220e+06  355612.772416   
1   고위험   안전  8.526810e+06  2.062608e+06  6.464203e+06  355642.795521   
2   고위험   위험  8.526810e+06  1.072814e+06  7.453997e+06  355619.762366   
3   고위험   이탈  8.526810e+06  4.900921e+04  8.477801e+06  355582.485966   
4   고위험  초안전  8.526810e+06  1.002450e+07 -1.497688e+06  466796.949666   
5   무관심   안전  7.615902e+05  2.062608e+06 -1.301018e+06    8117.763022   
6   무관심   위험  7.615902e+05  1.072814e+06 -3.112234e+05    7036.721933   
7   무관심   이탈  7.615902e+05  4.900921e+04  7.125810e+05    4796.289849   
8   무관심  초안전  7.615902e+05  1.002450e+07 -9.262908e+06  302462.381607   
9    안전   위험  2.062608e+06  1.072814e+06  9.897942e+05    8418.405796   
10   안전   이탈  2.062608e+06  4.900921e+04  2.013599e+06    6660.217481   
11   안전  초안전  2.062608e+06  1.002450e+07 -7.961890e+06  302497.679972   
12   위험   이탈  1.072814e+06  4.900921e+04  1.023804e

In [ ]:
# ANOVA 실행
f_value, p_value = f_oneway(
    df[df['이탈여부'] == '이탈']['수신_적립식예금'],
    df[df['이탈여부'] == '위험']['수신_적립식예금'],
    df[df['이탈여부'] == '무관심']['수신_적립식예금'],
    df[df['이탈여부'] == '안전']['수신_적립식예금'],
    df[df['이탈여부'] == '초안전']['수신_적립식예금'],
    df[df['이탈여부'] == '고위험']['수신_적립식예금'],
)

# 결과 출력
print(f"F-value: {f_value}, P-value: {p_value}")

# p-value에 따라 결론 내기
if p_value < 0.05:
    print("그룹 간 평균에 유의미한 차이가 있습니다.")
else:
    print("그룹 간 평균에 유의미한 차이가 없습니다.")

F-value: 119929.69416295896, P-value: 0.0
그룹 간 평균에 유의미한 차이가 있습니다.


In [272]:
# 안전/초안전
posthoc = pg.pairwise_gameshowell(dv='수신_적립식예금', between='이탈여부', data=df_12)
print(posthoc)

      A    B       mean(A)       mean(B)          diff             se  \
0   고위험  무관심  3.457436e+06  4.951599e+05  2.962276e+06  166082.987279   
1   고위험   안전  3.457436e+06  2.149964e+06  1.307472e+06  166107.354017   
2   고위험   위험  3.457436e+06  7.988925e+05  2.658543e+06  166089.644788   
3   고위험   이탈  3.457436e+06  2.665251e+04  3.430783e+06  166072.989781   
4   고위험  초안전  3.457436e+06  2.051630e+06  1.405806e+06  181254.382077   
5   무관심   안전  4.951599e+05  2.149964e+06 -1.654804e+06    3866.550131   
6   무관심   위험  4.951599e+05  7.988925e+05 -3.037327e+05    3011.188680   
7   무관심   이탈  4.951599e+05  2.665251e+04  4.685074e+05    1880.182483   
8   무관심  초안전  4.951599e+05  2.051630e+06 -1.556470e+06   72639.164184   
9    안전   위험  2.149964e+06  7.988925e+05  1.351071e+06    4142.662486   
10   안전   이탈  2.149964e+06  2.665251e+04  2.123311e+06    3410.202542   
11   안전  초안전  2.149964e+06  2.051630e+06  9.833416e+04   72694.859299   
12   위험   이탈  7.988925e+05  2.665251e+04  7.722400e

In [ ]:
# ANOVA 실행
f_value, p_value = f_oneway(
    df[df['이탈여부'] == '이탈']['수신_펀드'],
    df[df['이탈여부'] == '위험']['수신_펀드'],
    df[df['이탈여부'] == '무관심']['수신_펀드'],
    df[df['이탈여부'] == '안전']['수신_펀드'],
    df[df['이탈여부'] == '초안전']['수신_펀드'],
    df[df['이탈여부'] == '고위험']['수신_펀드'],
)

# 결과 출력
print(f"F-value: {f_value}, P-value: {p_value}")

# p-value에 따라 결론 내기
if p_value < 0.05:
    print("그룹 간 평균에 유의미한 차이가 있습니다.")
else:
    print("그룹 간 평균에 유의미한 차이가 없습니다.")

F-value: 3634.9710793844934, P-value: 0.0
그룹 간 평균에 유의미한 차이가 있습니다.


In [274]:
# 고위험/초안전, 안전/초안전

posthoc = pg.pairwise_gameshowell(dv='수신_펀드', between='이탈여부', data=df_12)
print(posthoc)

      A    B        mean(A)        mean(B)           diff            se  \
0   고위험  무관심  424337.689862   27985.447458  396352.242404  75926.663813   
1   고위험   안전  424337.689862  160571.983981  263765.705882  75939.739654   
2   고위험   위험  424337.689862   86687.898232  337649.791630  75933.564163   
3   고위험   이탈  424337.689862    1104.671777  423233.018086  75924.463208   
4   고위험  초안전  424337.689862  215691.776387  208645.913475  83263.102245   
5   무관심   안전   27985.447458  160571.983981 -132586.536522   1632.025252   
6   무관심   위험   27985.447458   86687.898232  -58702.450774   1313.626319   
7   무관심   이탈   27985.447458    1104.671777   26880.775682    586.140781   
8   무관심  초안전   27985.447458  215691.776387 -187706.328928  34184.260166   
9    안전   위험  160571.983981   86687.898232   73884.085748   1926.498141   
10   안전   이탈  160571.983981    1104.671777  159467.312204   1526.218200   
11   안전  초안전  160571.983981  215691.776387  -55119.792406  34213.293090   
12   위험   이탈   86687.8982

In [ ]:
# ANOVA 실행
f_value, p_value = f_oneway(
    df[df['이탈여부'] == '이탈']['수신_외화예금'],
    df[df['이탈여부'] == '위험']['수신_외화예금'],
    df[df['이탈여부'] == '무관심']['수신_외화예금'],
    df[df['이탈여부'] == '안전']['수신_외화예금'],
    df[df['이탈여부'] == '초안전']['수신_외화예금'],
    df[df['이탈여부'] == '고위험']['수신_외화예금'],
)

# 결과 출력
print(f"F-value: {f_value}, P-value: {p_value}")

# p-value에 따라 결론 내기
if p_value < 0.05:
    print("그룹 간 평균에 유의미한 차이가 있습니다.")
else:
    print("그룹 간 평균에 유의미한 차이가 없습니다.")

F-value: 459.5258519790997, P-value: 0.0
그룹 간 평균에 유의미한 차이가 있습니다.


In [276]:
# 고위험/무관심, 고위험/안전, 고위험/위험, 고위험/이탈, 무관심/초안전, 안전/초안전
posthoc = pg.pairwise_gameshowell(dv='수신_외화예금', between='이탈여부', data=df_12)
print(posthoc)

      A    B        mean(A)        mean(B)           diff            se  \
0   고위험  무관심  188943.836100    3485.701378  185458.134723  57041.355742   
1   고위험   안전  188943.836100   19000.191698  169943.644402  57043.762789   
2   고위험   위험  188943.836100   16952.312077  171991.524023  57044.180943   
3   고위험   이탈  188943.836100     322.475219  188621.360881  57040.825678   
4   고위험  초안전  188943.836100  156442.909562   32500.926538  70866.652814   
5   무관심   안전    3485.701378   19000.191698  -15514.490321    634.419334   
6   무관심   위험    3485.701378   16952.312077  -13466.610699    670.965173   
7   무관심   이탈    3485.701378     322.475219    3163.226159    259.631954   
8   무관심  초안전    3485.701378  156442.909562 -152957.208184  42053.467108   
9    안전   위험   19000.191698   16952.312077    2047.879621    851.353373   
10   안전   이탈   19000.191698     322.475219   18677.716479    584.822241   
11   안전  초안전   19000.191698  156442.909562 -137442.717863  42056.731971   
12   위험   이탈   16952.3120

In [ ]:
# ANOVA 실행
f_value, p_value = f_oneway(
    df[df['이탈여부'] == '이탈']['대출금액'],
    df[df['이탈여부'] == '위험']['대출금액'],
    df[df['이탈여부'] == '무관심']['대출금액'],
    df[df['이탈여부'] == '안전']['대출금액'],
    df[df['이탈여부'] == '초안전']['대출금액'],
    df[df['이탈여부'] == '고위험']['대출금액'],
)

# 결과 출력
print(f"F-value: {f_value}, P-value: {p_value}")

# p-value에 따라 결론 내기
if p_value < 0.05:
    print("그룹 간 평균에 유의미한 차이가 있습니다.")
else:
    print("그룹 간 평균에 유의미한 차이가 없습니다.")

F-value: 37163.13014140561, P-value: 0.0
그룹 간 평균에 유의미한 차이가 있습니다.


In [278]:
posthoc = pg.pairwise_gameshowell(dv='대출금액', between='이탈여부', data=df_12)
print(posthoc)

      A    B       mean(A)       mean(B)          diff             se  \
0   고위험  무관심  1.056196e+07  1.073020e+06  9.488937e+06  440550.367703   
1   고위험   안전  1.056196e+07  1.386252e+06  9.175705e+06  440556.633116   
2   고위험   위험  1.056196e+07  2.993199e+06  7.568758e+06  440603.410102   
3   고위험   이탈  1.056196e+07  1.839244e+05  1.037803e+07  440524.092865   
4   고위험  초안전  1.056196e+07  1.409683e+07 -3.534870e+06  575965.853276   
5   무관심   안전  1.073020e+06  1.386252e+06 -3.132318e+05    7454.867595   
6   무관심   위험  1.073020e+06  2.993199e+06 -1.920179e+06    9838.346493   
7   무관심   이탈  1.073020e+06  1.839244e+05  8.890955e+05    5186.951239   
8   무관심  초안전  1.073020e+06  1.409683e+07 -1.302381e+07  371082.325378   
9    안전   위험  1.386252e+06  2.993199e+06 -1.606947e+06   10115.016610   
10   안전   이탈  1.386252e+06  1.839244e+05  1.202327e+06    5694.292094   
11   안전  초안전  1.386252e+06  1.409683e+07 -1.271058e+07  371089.763679   
12   위험   이탈  2.993199e+06  1.839244e+05  2.809274e

In [ ]:
# 지역 대구경북/그외 이탈/비이탈 카이제곱(연관성) 성별?
df=df.drop_duplicates(subset='고객ID')

In [280]:
df[df['이탈여부'] == '이탈']['지역구분'].value_counts()

지역구분
1    110578
0     75469
Name: count, dtype: int64

In [281]:
df[~(df['이탈여부'] == '이탈')]['지역구분'].value_counts()

지역구분
1    299030
0     64164
Name: count, dtype: int64

In [282]:
df

,기준년월,고객ID,연령대,성별,고객등급,수신_요구불예금,수신_거치식예금,수신_적립식예금,수신_펀드,수신_외화예금,대출금액,총자산,분기,자산변화율,이탈여부,지역구분,대출여부
880,202312,9996691351,30대,남,일반등급,0,0,0,0,0,0,0,4,0.000000,이탈,1,0
881,202312,9996689486,30대,여,일반등급,23000000,0,0,0,0,0,23000000,4,-4.725183,위험,1,0
882,202312,9996683258,30대,여,일반등급,100000,0,0,0,0,0,100000,4,0.000000,무관심,1,0
883,202312,9996680650,30대,여,일반등급,0,0,0,0,0,0,0,4,0.000000,이탈,1,0
884,202312,9996676637,30대,여,일반등급,400000,0,0,0,0,0,400000,4,0.017501,안전,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21814931,202301,2458976512,30대,여,일반등급,0,0,0,0,0,0,0,1,0.017578,이탈,1,0
21815138,202301,2457998417,30대,여,일반등급,200000,0,0,0,0,200000,400000,1,0.650392,안전,0,1
21815271,202301,2457388357,30대,남,일반등급,0,0,0,0,0,100000,100000,1,0.298829,안전,0,1
21815547,202301,2456203527,30대,남,일반등급,0,0,0,0,0,100000,100000,1,0.439454,안전,0,1


In [283]:
df = {'이탈': df[df['이탈여부'] == '이탈']['지역구분'].value_counts(),
'비이탈':df[~(df['이탈여부'] == '이탈')]['지역구분'].value_counts()}

In [284]:
df = pd.DataFrame(df)

In [285]:
df

,이탈,비이탈
지역구분,,
1,110578,299030
0,75469,64164


In [286]:
# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(df)

# 결과 출력
print(f"Chi2 statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies: \n{expected}")

# p-value에 따라 결론 도출
if p < 0.05:
    print("두 변수 간에 연관이 있습니다. (귀무가설 기각)")
else:
    print("두 변수 간에 연관이 없습니다. (귀무가설 채택)")

Chi2 statistic: 34020.70308657563
P-value: 0.0
Degrees of freedom: 1
Expected frequencies: 
[[138748.45391367 270859.54608633]
 [ 47298.54608633  92334.45391367]]
두 변수 간에 연관이 있습니다. (귀무가설 기각)


In [287]:
df_sex=df_12.drop_duplicates(subset='고객ID')
df_sex = {'이탈': df_sex[df_sex['이탈여부'] == '이탈']['성별'].value_counts(),
'비이탈':df_sex[~(df_sex['이탈여부'] == '이탈')]['성별'].value_counts()}
df_sex = pd.DataFrame(df_sex)

In [288]:
# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(df_sex)

# 결과 출력
print(f"Chi2 statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies: \n{expected}")

# p-value에 따라 결론 도출
if p < 0.05:
    print("두 변수 간에 연관이 있습니다. (귀무가설 기각)")
else:
    print("두 변수 간에 연관이 없습니다. (귀무가설 채택)")

Chi2 statistic: 484.2548987982183
P-value: 2.534519353763797e-107
Degrees of freedom: 1
Expected frequencies: 
[[ 96050.26414998 187505.73585002]
 [ 89996.73585002 175688.26414998]]
두 변수 간에 연관이 있습니다. (귀무가설 기각)


In [289]:
# df_12.to_csv(f'../dataset/imbank_{TARGET.lower()}_data/{TARGET.lower()}_data_1112.csv', encoding='euc-kr', index=False)